<a href="https://colab.research.google.com/github/lazarnouche/SEM_generation_GAN/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
from google.colab import drive
from skimage import io
from skimage.transform import resize
import os
from google.colab import drive
from tensorflow.keras.datasets import mnist
drive.mount('/content/drive/')

Found GPU at: /device:GPU:0
Mounted at /content/drive/


In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
os.chdir('/content/drive/My Drive/Colab Notebooks')

In [ ]:
import numpy as np
from tensorflow.keras import layers,models,Input,optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from matplotlib import pyplot as plt
import os

latent_dim = 32
height = 32
width = 32
channels = 1

In [ ]:
generator_input = Input(shape=(latent_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = models.Model(generator_input, x)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       1048832   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 256)       0     

In [ ]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = models.Model(discriminator_input, x)
discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       1280      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 6, 6, 128)         0   

In [ ]:
discriminator_optimizer = optimizers.RMSprop( learning_rate=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

# The adversarial network

In [ ]:
discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
# generator(gan_input)
gan_output = discriminator(generator(gan_input))
gan = models.Model(gan_input, gan_output)
gan_optimizer = optimizers.RMSprop(learning_rate=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
gan.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
model (Functional)           (None, 32, 32, 1)         6239489   
_________________________________________________________________
model_1 (Functional)         (None, 1)                 788609    
Total params: 7,028,098
Trainable params: 6,239,489
Non-trainable params: 788,609
_________________________________________________________________


In [ ]:
# (x_train, y_train), (_, _) = tensorflow.keras.datasets.cifar10.load_data()
# x_train = x_train[y_train.flatten() == 6]
# x_train = x_train.reshape(
# (x_train.shape[0],) +
# (height, width, channels)).astype('float32') / 255.

In [ ]:
(X_train, _), (_, _) = mnist.load_data()
X_train = X_train / 127.5 - 1.
X_train = np.expand_dims(X_train, axis=3)

In [ ]:
X_train.shape

(60000, 28, 28, 1)

In [ ]:
iterations = 10000
batch_size = 20
save_dir = os.getcwd()

In [ ]:
sample_images(epoch,generator):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, latent_dim))
        gen_imgs = generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [ ]:
start = 0
for step in range(iterations):
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
  generated_images = generator.predict(random_latent_vectors)
  stop = start + batch_size
  real_images = X_train[start: stop]
  combined_images = np.concatenate([generated_images, real_images])
  labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
  labels += 0.05 * np.random.random(labels.shape) # add random noise
  d_loss = discriminator.train_on_batch(combined_images, labels)
  random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
  misleading_targets = np.zeros((batch_size, 1))
  a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
  start += batch_size
  if start > len(x_train) - batch_size:
    start = 0
  if step % 100 == 0:
    sample_images(epoch)
    gan.save_weights('gan.h5')
    print('discriminator loss:', d_loss)
    print('adversarial loss:', a_loss)
    sample_images(epoch,generator)
    # plt.imshow(image.array_to_img(generated_images[0] * 255., scale=False))
    # plt.imshow(image.array_to_img(real_images[0] * 255., scale=False))



ValueError: ignored

ValueError: ignored

In [ ]:
plt.imshow(image.array_to_img(generated_images[0] * 255., scale=False))
plt.figure()
plt.imshow(image.array_to_img(real_images[0] * 255., scale=False))

NameError: ignored

In [ ]:
  combined_images = np.concatenate([generated_images, real_images])


# Zeolite

In [ ]:
def return_data(file,img_size=80):
    X=[]
    y=[]
    for key,value in file.items():
        for v in value:
            img = io.imread(v)
            img=resize(img, (img_size, img_size))
            X.append(img)
    return np.array(X).reshape(-1, img_size, img_size, 3)

In [ ]:
directory = os.getcwd()
train_dir = os.path.join(directory,'train')
validation_dir = os.path.join(directory,'validation')
file={}

data_paths = lambda directory : [os.path.join(os.path.join(directory,face),
                            sub) for sub in os.listdir(os.path.join(directory,face)) if os.path.splitext(sub)[1]=='.png']
for face in ['1Face', '2Faces','3Faces']:
  file[face] = data_paths(train_dir)+data_paths(validation_dir)


In [ ]:
latent_dim = 64
height = 64
width = 64
channels = 3

In [ ]:
X_train = return_data(file,img_size=height)

In [ ]:
X_train.shape

(265, 64, 64, 3)

In [ ]:
x_train = X_train.reshape(
(X_train.shape[0],) +
(height, width, channels)).astype('float32') / 255.

In [ ]:
generator_input = Input(shape=(latent_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = models.Model(generator_input, x)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             2129920   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       1048832   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 256)       0     

In [ ]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = models.Model(discriminator_input, x)
discriminator_optimizer = optimizers.RMSprop( learning_rate=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')
discriminator.summary()

In [ ]:
discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
generator(gan_input)
gan_output = discriminator(generator(gan_input))
gan = models.Model(gan_input, gan_output)
gan_optimizer = optimizers.RMSprop(learning_rate=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
gan.summary()

In [ ]:
x_train[start: stop].shape

In [ ]:
iterations = 1000
batch_size =20
start = 0
save_dir = os.getcwd()
for step in range(iterations):
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
  generated_images = generator.predict(random_latent_vectors)
  stop = start + batch_size
  real_images = x_train[start: stop]
  combined_images = np.concatenate([generated_images, real_images])
  labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
  labels += 0.05 * np.random.random(labels.shape) # add random noise
  d_loss = discriminator.train_on_batch(combined_images, labels)
  random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
  misleading_targets = np.zeros((batch_size, 1))
  a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
  start += batch_size
  if start > len(x_train) - batch_size:
    start = 0
  if step % 100 == 0:
    gan.save_weights('gan.h5')
    print('discriminator loss:', d_loss)
    print('adversarial loss:', a_loss)
    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,'generated_zeo' + str(step) + '.png'))
    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,'real_zeo' + str(step) + '.png'))
